## Radarbanten.co.id Search 

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import threading
import re

In [2]:
jumlah_index = 41
threads_link = []
links = []
results = []
threads = []
keywords = 'ganjar'

In [3]:
def scrape_links(page_number,keywords):
    url=f"https://www.radarbanten.co.id/page/{page_number}/?s={keywords}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('h3',{"class": "jeg_post_title"})
    
    page_links = []
    for article in articles:
        link = article.find('a')['href']
        page_links.append(link)
    
    print(f"Scraped {len(page_links)} links from page {page_number}")
    return page_links

In [4]:
for page_number in range(1, jumlah_index + 1):
    thread = threading.Thread(target=lambda p=page_number: links.extend(scrape_links(p,keywords)))
    thread.start()
    threads_link.append(thread)

for thread in threads_link:
    thread.join()
print("Total Links:", len(links))

Scraped 18 links from page 12
Scraped 18 links from page 10
Scraped 18 links from page 36
Scraped 18 links from page 30
Scraped 18 links from page 22
Scraped 18 links from page 15
Scraped 18 links from page 31
Scraped 18 links from page 33
Scraped 18 links from page 8
Scraped 18 links from page 5
Scraped 18 links from page 38
Scraped 18 links from page 6
Scraped 18 links from page 34
Scraped 18 links from page 16
Scraped 18 links from page 19
Scraped 18 links from page 40
Scraped 18 links from page 13
Scraped 18 links from page 32
Scraped 18 links from page 20
Scraped 18 links from page 29
Scraped 18 links from page 1
Scraped 18 links from page 37
Scraped 18 links from page 3
Scraped 18 links from page 39
Scraped 18 links from page 24
Scraped 18 links from page 21
Scraped 18 links from page 26
Scraped 9 links from page 41
Scraped 18 links from page 35
Scraped 18 links from page 14
Scraped 18 links from page 25
Scraped 18 links from page 11
Scraped 18 links from page 7
Scraped 18 links 

In [5]:
def scrape_url(url,keywords):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')        
            # Judul Berita
            title_elem = soup.find('h1')
            if title_elem:
                title_text = title_elem.text
            else:
                title_text = "Title not found"
            # Author berita
            author_elem = soup.find('div', {"class": "jeg_meta_author"})
            if author_elem:
                author_text = author_elem.find('a')
                author_text = author_text.text
            else:
                author_text = "Author not found"
                     
            # tanggal berita
            date_elem = soup.find('div', {"class": "jeg_meta_date"})
            if date_elem:
                date_text = date_elem.find('a')
                date_text = date_text.text
            else:
                date_text = "Date not found"
            # Category berita
            category_elements = soup.find('span', {"class": "breadcrumb_last_link"})
            if category_elements:
                category_text = category_elements.find('a')
                category_text= category_text.text
            else:
                category_text = "Category not found"
            #     # Content Berita
            body_elem = soup.find('div', {"class": "entry-content with-share"})
            
            if body_elem:
                content_elem = body_elem.find_all('p')
                content_text = ""
                for p in content_elem:
                    content_text += p.text.strip() + "\n"
                
                if content_text.strip():
                    content_text=content_text
                else:
                    content_text ="Content not found"
            else:
                content_text ="Content not found"
            nama_berita_match = re.search(r'https://www\.(\w+)\.co\.id/', url)
            if nama_berita_match:
                nama_berita = nama_berita_match.group(1)
            else:
                nama_berita = "Nama_berita not found"


            results.append({'title': title_text,
                            'keywords':keywords,
                            'author' : author_text,
                            'category':category_text,
                            'date': date_text,
                            'content' : content_text,
                            'nama_berita' : nama_berita,
                            'region':'banten',
                            'link' : url})
        else:
            print(f"Failed to retrieve data from {url}")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL '{url}': {e}")
    except Exception as e:
        print(f"Error processing URL '{url}': {e}")

In [6]:
for url in links:
    thread = threading.Thread(target=scrape_url, args=(url,keywords))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

In [7]:
df = pd.DataFrame(results)
print("Total Hasil ",len(results))
df.head(10)

Total Hasil  729


,title,keywords,author,category,date,content,nama_berita,region,link
0,Megawati Samakan Rano dengan Risma dan Ganjar,ganjar,Redaksi,Berita Utama,04-02-2017 15:01:03,TANGERANG – Ketua Umum PDIP Megawati Soekarno ...,radarbanten,banten,https://www.radarbanten.co.id/megawati-samakan...
1,LKBA Kabupaten Serang Dapat Selesaikan Permasa...,ganjar,Ahmad Rizal Ramdhani,Kabupaten Serang,12-10-2023 16:56:58,"SERANG, RADARBANTEN.CO.ID – Pemerintah Kabupat...",radarbanten,banten,https://www.radarbanten.co.id/lkba-kabupaten-s...
2,"Efektif Pantau Harga, Gubernur Ganjar Silahkan...",ganjar,Aas Arbi,Pemerintahan,20-11-2017 16:38:38,JAKARTA – Gubernur Jawa Tengah Ganjar Pranowo ...,radarbanten,banten,https://www.radarbanten.co.id/efektif-pantau-h...
3,"Peras Pengembang Rp 530 Juta, Mantan Kades dan...",ganjar,Fahmi,Hukum,05-10-2023 20:23:42,"SERANG, RADARBANTEN.CO.ID – Mantan Kepala Desa...",radarbanten,banten,https://www.radarbanten.co.id/peras-pengembang...
4,"Terkait Tes CPNS 2014, BKD Cilegon Diganjar Pe...",ganjar,Redaksi,Pemerintahan,13-03-2015 16:16:13,CILEGON – Badan Kepegawaian dan Diklat (BKD) K...,radarbanten,banten,https://www.radarbanten.co.id/terkait-tes-cpns...
5,"Jorok, 240.402 Keluarga di Banten Tak Punya Ja...",ganjar,Rostinah,Kesehatan,10-10-2023 16:02:58,"SERANG, RADARBANTEN.CO.ID – Sebanyak 240.402 k...",radarbanten,banten,https://www.radarbanten.co.id/jorok-240-402-ke...
6,Dadang Prijatna Diganjar 4 Tahun Penjara Terka...,ganjar,Aas Arbi,Hukum,26-10-2015 16:25:51,Majlis hakim yang dipimpin oleh Jesden Purba m...,radarbanten,banten,https://www.radarbanten.co.id/dadang-prijatna-...
7,Pemkab Serang Siapkan 2.540 Loker di Job Fair ...,ganjar,Ahmad Rizal Ramdhani,Kabupaten Serang,11-10-2023 15:37:54,"SERANG,RADARBANTEN.CO.ID- Pemerintah Kabupaten...",radarbanten,banten,https://www.radarbanten.co.id/pemkab-serang-si...
8,Elektabilitas Prabowo di Banten Disalip Ganjar...,ganjar,Yusuf Permana,Politik,12-10-2023 16:42:56,"SERANG, RADARBANTEN.CO.ID – DPD Gerindra Bante...",radarbanten,banten,https://www.radarbanten.co.id/elektabilitas-pr...
9,Kejari Cilegon Akan Pelajari Lapdu Dugaan Pela...,ganjar,Bayu Mulyana,Hukum,11-10-2023 17:31:53,"CILEGON,RADARBANTEN.CO.ID- Kejaksaan Negeri (K...",radarbanten,banten,https://www.radarbanten.co.id/kejari-cilegon-a...


In [8]:
filtered_df = df[df['content'].str.contains(keywords, case=False)|
                 df['title'].str.contains(keywords, case=False)]
print('hasil filter',len(filtered_df))
filtered_df.head(10)

hasil filter 410


,title,keywords,author,category,date,content,nama_berita,region,link
0,Megawati Samakan Rano dengan Risma dan Ganjar,ganjar,Redaksi,Berita Utama,04-02-2017 15:01:03,TANGERANG – Ketua Umum PDIP Megawati Soekarno ...,radarbanten,banten,https://www.radarbanten.co.id/megawati-samakan...
2,"Efektif Pantau Harga, Gubernur Ganjar Silahkan...",ganjar,Aas Arbi,Pemerintahan,20-11-2017 16:38:38,JAKARTA – Gubernur Jawa Tengah Ganjar Pranowo ...,radarbanten,banten,https://www.radarbanten.co.id/efektif-pantau-h...
3,"Peras Pengembang Rp 530 Juta, Mantan Kades dan...",ganjar,Fahmi,Hukum,05-10-2023 20:23:42,"SERANG, RADARBANTEN.CO.ID – Mantan Kepala Desa...",radarbanten,banten,https://www.radarbanten.co.id/peras-pengembang...
4,"Terkait Tes CPNS 2014, BKD Cilegon Diganjar Pe...",ganjar,Redaksi,Pemerintahan,13-03-2015 16:16:13,CILEGON – Badan Kepegawaian dan Diklat (BKD) K...,radarbanten,banten,https://www.radarbanten.co.id/terkait-tes-cpns...
6,Dadang Prijatna Diganjar 4 Tahun Penjara Terka...,ganjar,Aas Arbi,Hukum,26-10-2015 16:25:51,Majlis hakim yang dipimpin oleh Jesden Purba m...,radarbanten,banten,https://www.radarbanten.co.id/dadang-prijatna-...
8,Elektabilitas Prabowo di Banten Disalip Ganjar...,ganjar,Yusuf Permana,Politik,12-10-2023 16:42:56,"SERANG, RADARBANTEN.CO.ID – DPD Gerindra Bante...",radarbanten,banten,https://www.radarbanten.co.id/elektabilitas-pr...
10,"Diganjar Disclamer, Rano Sebut Warisan Masa La...",ganjar,Redaksi,Pemerintahan,01-06-2015 18:24:29,SERANG – Plt Gubernur Banten Rano Karno berjan...,radarbanten,banten,https://www.radarbanten.co.id/diganjar-disclam...
11,Santri Milenial Banten Deklarasikan Gibran for...,ganjar,Yusuf Permana,Utama,09-10-2023 16:34:41,"SERANG, RADARBANTEN.CO.ID – Ratusan santri di ...",radarbanten,banten,https://www.radarbanten.co.id/santri-milenial-...
12,Soal Kemunculan Dewan Kopral Dan Dewan Kolonel...,ganjar,Redaksi,Utama,23-09-2022 15:20:13,"SERANG, RADARBANTEN.CO.ID – Gubernur Jawa Teng...",radarbanten,banten,https://www.radarbanten.co.id/soal-kemunculan-...
15,Pembunuh Perajin Tas Diganjar Tujuh Tahun Penjara,ganjar,Redaksi,Berita Utama,25-11-2016 11:59:49,SERANG – Zwesda alias Uwes (53) divonis tujuh ...,radarbanten,banten,https://www.radarbanten.co.id/pembunuh-perajin...


# Save File 

In [9]:
current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
excel_file_name = f'../../tempat_hasil_daerah/radarBanten_{keywords}_{current_datetime}.xlsx'
filtered_df.to_excel(excel_file_name, index=False)

print(f'Data has been saved to {excel_file_name}')

Data has been saved to ../../tempat_hasil_daerah/radarBanten_ganjar_2023-10-12_17-22-12.xlsx
